In [ ]:
%%capture
!pip install cvlib

In [1]:
%cd /content
!gdown --id 13nvlIOMwfSFCSRdO-h_YNnfpGwhCwPXa
!gdown --id 1u_tgBn4BR5mPjCSBxv-yBQc9y2cQwP52

/content
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13nvlIOMwfSFCSRdO-h_YNnfpGwhCwPXa
To: /content/model.h5
100% 182M/182M [00:03<00:00, 49.7MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1u_tgBn4BR5mPjCSBxv-yBQc9y2cQwP52
To: /content/weights.h5
100% 91.0M/91.0M [00:01<00:00, 51.9MB/s]


In [3]:
data_path = '/content/drive/MyDrive/Colab Notebooks/AiffelThon/gender_classification/'

In [8]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

# model path
model_path = "model.h5"
model_weights_path = "weights.h5"

image_path = data_path + '4.png'
im = cv2.imread(image_path)

# load model
model = load_model(model_path)
model.load_weights(model_weights_path)


faces, confidences = cv.detect_face(im)
# loop through detected faces and add bounding box
for face in faces:
    (startX,startY) = face[0],face[1]
    (endX,endY) = face[2],face[3]
    # draw rectangle over face
    cv2.rectangle(im, (startX,startY), (endX,endY), (0,100,0), 2)

    # preprocessing for gender detection model
    cropped_face = im[startY:endY,startX:endX]
    cropped_face = cv2.resize(cropped_face, (150,150))
    cropped_face = cropped_face.astype("float32") / 255
    cropped_face = img_to_array(cropped_face)
    cropped_face = np.expand_dims(cropped_face, axis=0)

    # apply model
    conf = model.predict(cropped_face)[0]

    if conf[0] > conf[1]:
        if conf[0] - conf[1] < conf[1]:
            label = "Maybe male"
        else:
            label = "Male"
    else:
        if conf[0] < conf[1]:
            if conf[1] - conf[0] < conf[0]:
                label = "Maybe female"
            else:
                label = "Female"

    print(conf)
    print(label)

    if label.find("Maybe") >= 0: 
        cv2.putText(im, label, (startX, startY-5),  cv2.FONT_HERSHEY_SIMPLEX,0.5, (200,0,0), 2)
    else:
        cv2.putText(im, label, (startX, startY-5),  cv2.FONT_HERSHEY_SIMPLEX,1, (200,0,0), 2) 

[0.80704564 0.19295436]
Male


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 activation_1 (Activation)   (None, 150, 150, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        8256      
                                                                 
 activation_2 (Activation)   (None, 75, 75, 64)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 37, 32)       0         
 2D)                                                  